In [6]:
import csv
import numpy as np
from pprint import pprint

In [11]:
# Threshold coefficient.
m = 0.5

In [29]:
def read_data(file_path):
    with open(file_path, 'rb') as csvfile:
        csvreader = csv.reader(csvfile)
        features = csvreader.next()
        data = []
        for row in csvreader:
            data.append(row)
    return {'features': features, 'data': np.array(data)}

input_data = read_data('dataset_diabetes/diabetic_data.csv')
features = input_data['features']
data = np.array(input_data['data'])
N, V = data.shape
pprint(input_data)

{'data': array([['2278392', '8222157', 'Caucasian', ..., 'No', 'No', 'NO'],
       ['149190', '55629189', 'Caucasian', ..., 'Ch', 'Yes', '>30'],
       ['64410', '86047875', 'AfricanAmerican', ..., 'No', 'Yes', 'NO'],
       ...,
       ['443854148', '41088789', 'Caucasian', ..., 'Ch', 'Yes', 'NO'],
       ['443857166', '31693671', 'Caucasian', ..., 'Ch', 'Yes', 'NO'],
       ['443867222', '175429310', 'Caucasian', ..., 'No', 'No', 'NO']],
      dtype='|S36'),
 'features': ['encounter_id',
              'patient_nbr',
              'race',
              'gender',
              'age',
              'weight',
              'admission_type_id',
              'discharge_disposition_id',
              'admission_source_id',
              'time_in_hospital',
              'payer_code',
              'medical_specialty',
              'num_lab_procedures',
              'num_procedures',
              'num_medications',
              'number_outpatient',
              'number_emergency',
    

In [40]:
def calc_stats_i(data_i, V):
    mu_i = []
    mu_tilde_i = []
    sample_var_i = []
    selection_threshold_i = []
    
    # Loop over dimensions.
    for j in range(V):
        data_ij = data_i[:, j]
        mu_ij = np.mean(data_ij)
        mu_tilde_ij = np.median(data_ij)
        sample_var_ij = np.var(data_ij, ddof=1)
        selection_threshold_ij = m * sample_var

        mu_i.append(mu_ij)
        mu_tilde_i.append(mu_tilde_ij)
        sample_var_i.append(sample_var_ij)
        selection_threshold_i.append(selection_threshold_ij)
    return (mu_i, mu_tilde_i, sample_var_i, selection_threshold_i)

In [41]:
def calc_stats(data, C, V):
    # Calculate sample median, sample mean, sample variance and selection threshold.
    mu = []
    mu_tilde = []
    sample_var = []
    selection_threshold = []
    
    # Loop over clusters.
    for i in C:
        data_i = data[i]
        mu_i, mu_tilde_i, sample_var_i, selection_threshold_i = calc_stats_i(data_i, V)
        mu.append(mu_i)
        mu_tilde.append(mu_tilde_i)
        sample_var.append(sample_var_i)
        selection_threshold.append(selection_threshold_i)
        
    return {'mu': mu, 
            'mu_tilde': mu_tilde, 
            'sample_var': sample_var, 
            'selection_threshold': selection_threshold}

In [42]:
def select_dim(data_i, V, mu_hat_i=None):
    '''
    Select relevant dimensions for clusters, which must obey: 
    s[i][j] ** 2 + (mu[i][j] - mu_tilde[i][j]) ** 2 < s_hat[i][j] ** 2
    '''
    # Reture a list of 0/1 values to show whether dim_j is selected.
    selected_dim = np.zeros(V)
    
    # Calculate relevant statistics.
    mu_i, mu_tilde_i, sample_var_i, selection_threshold_i = calc_stats_i(V, data_i)
    
    # Use medoids if given.
    if mu_hat_i is not None:
        mu_tilde_i = mu_hat_i
    
    # Loop over clusters to find relevant dimensions.
    for j in range(V):
        if sample_var_i[j] ** 2 + (mu_i[j] - mu_tilde_i[j]) ** 2 < selection_threshold_i[j] ** 2:
            selected_dim[j] = 1
    return selected_dim

In [43]:
C = [[1, 2, 3], [9, 39, 43, 341], [32, 21, 432]]
calc_stats(data, C, V)

['149190' '64410' '500364']


TypeError: cannot perform reduce with flexible type